<a href="https://colab.research.google.com/github/iamchetry/Stock-Price-Prediction/blob/main/migrate_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
!pip install yfinance

import yfinance as yf
import datetime
import pandas as pd
import pytz
from google.cloud import bigquery
from google.colab import drive
import os
drive.mount('/content/drive')

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/drive/My Drive/stock-data-analysis-cse560-b3fc65360aa2.json'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
# Fetch stock data from Yahoo Finance
symbols = ['MSFT', 'GOOGL']
data_ = list()

for symbol in symbols:
    msft = yf.Ticker(symbol)
    df_hist = msft.history(period='max')
    df_hist.reset_index(level=0, inplace=True)
    df_hist.insert(0, "Symbol", symbol, True)
    df_hist.insert(0, "Market", 'NYSE', True)
    data_.append(df_hist)

data_ = pd.concat(data_, ignore_index=True, axis=0, sort=True)
data_ = data_[['Market', 'Symbol', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']].rename(columns={'Stock Splits': 'Stock_Splits'})

In [31]:
# Construct a BigQuery client object.
client = bigquery.Client()

# TODO(developer): Set table_id to the ID of the table to create.
table_id = 'stock-data-analysis-cse560.StockInfoDatabase.stock_history'

records = data_.to_dict(orient='records')

dataframe = pandas.DataFrame(
    records,
    # In the loaded table, the column order reflects the order of the
    # columns in the DataFrame.
    columns=list(data_.columns),
    # Optionally, set a named index, which can also be written to the
    # BigQuery table.
    index=pandas.Index(range(len(data_)), name='stock_ohlcv_data'
    ),
)

job_config = bigquery.LoadJobConfig(
    # Specify a (partial) schema. All columns are always written to the
    # table. The schema is used to assist in data type definitions.
    schema=[
        bigquery.SchemaField('Market', bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField('Symbol', bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField('stock_ohlcv_data', bigquery.enums.SqlTypeNames.INTEGER),
    ],
    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    write_disposition='WRITE_TRUNCATE',
)

job = client.load_table_from_dataframe(
    dataframe, table_id, job_config=job_config
)  # Make an API request.
job.result()  # Wait for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
    'Loaded {} rows and {} columns to {}'.format(
        table.num_rows, len(table.schema), table_id
    )
)

Loaded 13284 rows and 11 columns to stock-data-analysis-cse560.StockInfoDatabase.stock_history
